In [1]:
from dygie.commands.generate_metrics import load_documents, update_documents_with_clusters, update_documents_with_spans, load_jsonl
from collections import namedtuple

from collections import namedtuple
def convert(dictionary):
    return namedtuple('GenericDict', dictionary.keys())(**dictionary)

In [21]:
train_documents = load_documents('../model_data/pwc_split_on_sectioned/train.jsonl')

In [28]:
ner = []
import re
for d in train_documents :
    for e in train_documents[d]['ner'] :
        ner.append(re.sub(r'\W+', '', "".join(train_documents[d]['words'][e[0]:e[1]])))
ner = set(ner)

In [29]:
len(ner)

36810

In [4]:
documents = load_documents('../model_data/pwc_split_on_sectioned/dev.jsonl')
documents = update_documents_with_spans(documents, '../outputs/dev_outputs/spans.jsonl', convert({'gold' : False}))
documents = update_documents_with_clusters(documents, '../outputs/dev_outputs/clusters.jsonl')

\begin{tabular}{lrrrrrr}
\toprule
{} &  Exact F1 &  Exact P &  Exact R &  Soft F1 &  Soft P &  Soft R \\
\midrule
Method   &     0.784 &    0.779 &    0.789 &    0.808 &   0.803 &   0.813 \\
Task     &     0.707 &    0.705 &    0.709 &    0.739 &   0.738 &   0.741 \\
Metric   &     0.741 &    0.741 &    0.741 &    0.770 &   0.769 &   0.771 \\
Material &     0.562 &    0.553 &    0.570 &    0.602 &   0.591 &   0.613 \\
\bottomrule
\end{tabular}

macro_p             0.146962
macro_r             0.389234
macro_overlap_p     0.338188
macro_overlap_r     0.624775
macro_f1            0.213364
macro_overlap_f1    0.438836
dtype: float64


In [5]:
s = []
for d in documents :
    s.append(len(documents[d]['coref']))

In [6]:
sum(s)/len(s)

5.454545454545454

In [7]:
for d in documents :
    print(d)
    print(set(list(documents[d]['coref'].keys())))
    print(set([x for k,v in documents[d]['intersection_scores'].items() for x, _ in v.items()]))
    print("="*20)

02567fd428a675ca91a0c6786f47f3e35881bcbd
{'Age_Estimation', 'MORPH_Album2', 'MAE', 'ChaLearn_2015', 'DLDL', 'VGG-Face'}
{'DLDL', 'Age_Estimation', 'ChaLearn_2015', 'MAE'}
02b3d1d162080d9aefd3fc30a0bcc9a843073b5d
{'LSTM-8192-1024___CNN_Input', 'LSTM-8192-1024', 'CNN_Input', 'One_Billion_Word', 'Language_Modelling'}
{'One_Billion_Word'}
0398552184f80db111e9c28bf533b395f233ac00
{'Self-paced_curriculum_learning', 'Weakly_Supervised_Object_Detection', 'PASCAL_VOC_2007'}
{'PASCAL_VOC_2007', 'Weakly_Supervised_Object_Detection'}
05d2700846c0323f79c1344aca5333994c7c03a5
{'swb_hub_500_WER_fullSWBCH', '_model_M_', 'VGG', 'Speech_Recognition', 'CH__N-gram', 'Percentage_error', 'LSTM_acoustic_model_trained_on_SWB', 'NNLM_language_model', 'RNN', 'Switchboard___Hub500', 'RNN___VGG___LSTM_acoustic_model_trained_on_SWB_Fisher_CH__N-gram____model_M____NNLM_language_model'}
{'Percentage_error', 'Switchboard___Hub500', 'Speech_Recognition', 'swb_hub_500_WER_fullSWBCH'}
0626908dd710b91aece1a81f4ca0635f23f

In [5]:
n_clusters_clustering = []
for d in documents :
    n_clusters_clustering.append(len(documents[d]['predicted_clusters']))

In [30]:
data = []
data_ner = []
for d in documents :
    print(d)
    linked = set([x for k in documents[d]['linked_clusters'] for x, _ in documents[d]['intersection_scores'][k].items()])
    for k, v in documents[d]['coref'].items() :
        print(k, len(v), k in linked)
        data.append({'len' : len(v), 'linked' : k in linked})
        data_ner.append({'linked' : k in linked, 'in_train' : re.sub(r'\W+', '', k) in ner})
#     print(set(list(documents[d]['coref'].keys())))
#     print(set([(x, len(documents[d]['coref'][x])) for k,v in documents[d]['intersection_scores'].items() for x, _ in v.items()]))
    print("="*20)

02567fd428a675ca91a0c6786f47f3e35881bcbd
Age_Estimation 48 True
ChaLearn_2015 21 True
DLDL 84 True
MAE 20 True
MORPH_Album2 21 True
VGG-Face 4 False
02b3d1d162080d9aefd3fc30a0bcc9a843073b5d
CNN_Input 2 False
LSTM-8192-1024 38 False
LSTM-8192-1024___CNN_Input 35 False
Language_Modelling 37 True
One_Billion_Word 6 False
0398552184f80db111e9c28bf533b395f233ac00
PASCAL_VOC_2007 2 True
Self-paced_curriculum_learning 29 True
Weakly_Supervised_Object_Detection 41 True
05d2700846c0323f79c1344aca5333994c7c03a5
CH__N-gram 6 False
LSTM_acoustic_model_trained_on_SWB 18 False
NNLM_language_model 6 False
Percentage_error 4 True
RNN 20 False
RNN___VGG___LSTM_acoustic_model_trained_on_SWB_Fisher_CH__N-gram____model_M____NNLM_language_model 1 False
Speech_Recognition 6 True
Switchboard___Hub500 12 True
VGG 4 False
_model_M_ 5 False
swb_hub_500_WER_fullSWBCH 6 True
0626908dd710b91aece1a81f4ca0635f23fc47f3
ImageNet 8 True
Image_Classification 2 False
Inception_V3 23 True
Top_1_Accuracy 14 True
Top_5_Accu

In [55]:
doc = documents[list(documents.keys())[1]]

In [56]:
for p, v in doc['predicted_clusters'].items() :
    print([" ".join(doc['words'][x[0]:x[1]]) for x in v])
    print(p in doc['linked_clusters'])

['NLP', 'NLP', 'NLP tasks', 'NLP', 'NLP problems']
False
['character level CNNs', 'character embedding CNNs', 'character CNNs']
False
['low perplexity LMs', 'perplexity']
False
['single layer LSTM', 'single - layer LSTM of 1024 units']
False
['language modeling', 'Language Modeling', 'language modeling', 'neural language modeling', 'language modeling', 'language modeling', 'statistical language modeling', 'language modeling', 'Language Modeling']
True
['question answering', 'inference', 'inference', 'inference']
False
['One Billion Word Benchmark', 'One Billion Word Benchmark data set', 'One Billion Word Benchmark', '1B Word Benchmark data set', '1B Word Benchmark data set']
False
['CNN character embeddings', 'character - level embeddings', 'character - level embeddings']
False
['word error rate', 'IS', 'IS', 'IS', 'IS', 'IS']
True
['dropout', 'dropout', 'dropout']
False
['Recurrent Neural Networks', 'Recurrent Neural Networks', 'Recurrent Neural Networks', 'recurrent networks']
False


In [57]:
doc['doc_id']

'02b3d1d162080d9aefd3fc30a0bcc9a843073b5d'